In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [2]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [3]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)

In [10]:
df['Date']

0        2018-10-16
1        2018-10-16
2        2018-10-16
3        2018-10-16
4        2018-10-17
            ...    
33777    2024-04-06
33778    2024-04-06
33779    2024-04-06
33780    2024-04-06
33781    2024-04-06
Name: Date, Length: 33782, dtype: object

In [11]:
date_prediction = '2024-04-07'
df = df[df['Date'] == '2024-04-06']
df['date_next'] = add_col(df, 'Date')
df['date_next'].fillna(date_prediction, inplace=True)
dates = df['date_next'].unique()

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48042/138724945.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48042/3975340530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_next'] = add_col(df, 'Date')
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_48042/3975340530.py:4: F

In [12]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [13]:
# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 7:00 PM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

In [14]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
# adjust number of dates 
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings"]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

In [15]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

In [16]:
df

,Timestamp,Id,Sportsbook,Team,Odds
0,2024-04-07T22:30:38Z,d257ef9200525ae1a09743eeb8a2f90c,DraftKings,Dallas Mavericks,1.01
1,2024-04-07T22:30:38Z,d257ef9200525ae1a09743eeb8a2f90c,DraftKings,Houston Rockets,21.00
2,2024-04-07T22:30:38Z,d257ef9200525ae1a09743eeb8a2f90c,BetMGM,Dallas Mavericks,1.01
3,2024-04-07T22:30:38Z,d257ef9200525ae1a09743eeb8a2f90c,BetMGM,Houston Rockets,34.00
4,2024-04-07T22:30:38Z,c42ec787f2f26bbbd665aefd40337733,DraftKings,Indiana Pacers,1.04
...,...,...,...,...,...
65,2024-04-07T22:30:38Z,ad24e33d6da51ae673f54502c2dd8a23,DraftKings,Minnesota Timberwolves,1.91
66,2024-04-07T22:30:38Z,ad24e33d6da51ae673f54502c2dd8a23,FanDuel,Los Angeles Lakers,1.89
67,2024-04-07T22:30:38Z,ad24e33d6da51ae673f54502c2dd8a23,FanDuel,Minnesota Timberwolves,1.96
68,2024-04-07T22:30:38Z,ad24e33d6da51ae673f54502c2dd8a23,BetMGM,Los Angeles Lakers,1.87


In [17]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


['d257ef9200525ae1a09743eeb8a2f90c']

In [18]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [19]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

In [20]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-07T22:30:38Z,08d21ecc524683bf973f71d3b9264023,DraftKings,Milwaukee Bucks,1.56,New York Knicks,2.50
1,2024-04-07T22:30:38Z,08d21ecc524683bf973f71d3b9264023,DraftKings,New York Knicks,2.50,Milwaukee Bucks,1.56
2,2024-04-07T22:30:38Z,08d21ecc524683bf973f71d3b9264023,FanDuel,Milwaukee Bucks,1.56,New York Knicks,2.52
3,2024-04-07T22:30:38Z,08d21ecc524683bf973f71d3b9264023,FanDuel,New York Knicks,2.52,Milwaukee Bucks,1.56
4,2024-04-07T22:30:38Z,08d21ecc524683bf973f71d3b9264023,BetMGM,Milwaukee Bucks,1.53,New York Knicks,2.55
...,...,...,...,...,...,...,...
67,2024-04-07T22:30:38Z,ffff0c072d3db65180224daa6e216ca9,FanDuel,San Antonio Spurs,2.52,Philadelphia 76ers,1.56
68,2024-04-07T22:30:38Z,ffff0c072d3db65180224daa6e216ca9,DraftKings,Philadelphia 76ers,1.54,San Antonio Spurs,2.54
69,2024-04-07T22:30:38Z,ffff0c072d3db65180224daa6e216ca9,DraftKings,San Antonio Spurs,2.54,Philadelphia 76ers,1.54
70,2024-04-07T22:30:38Z,ffff0c072d3db65180224daa6e216ca9,BetMGM,Philadelphia 76ers,1.54,San Antonio Spurs,2.50


In [21]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [22]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [23]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-07,08d21ecc524683bf973f71d3b9264023,DraftKings,MIL,1.56,NYK,2.50
1,2024-04-07,08d21ecc524683bf973f71d3b9264023,DraftKings,NYK,2.50,MIL,1.56
2,2024-04-07,08d21ecc524683bf973f71d3b9264023,FanDuel,MIL,1.56,NYK,2.52
3,2024-04-07,08d21ecc524683bf973f71d3b9264023,FanDuel,NYK,2.52,MIL,1.56
4,2024-04-07,08d21ecc524683bf973f71d3b9264023,BetMGM,MIL,1.53,NYK,2.55
...,...,...,...,...,...,...,...
67,2024-04-07,ffff0c072d3db65180224daa6e216ca9,FanDuel,SAS,2.52,PHI,1.56
68,2024-04-07,ffff0c072d3db65180224daa6e216ca9,DraftKings,PHI,1.54,SAS,2.54
69,2024-04-07,ffff0c072d3db65180224daa6e216ca9,DraftKings,SAS,2.54,PHI,1.54
70,2024-04-07,ffff0c072d3db65180224daa6e216ca9,BetMGM,PHI,1.54,SAS,2.50


In [24]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [25]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [26]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [27]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2024-04-07,MIL,1.56,1.56,1.53,NYK,2.52,2.50,2.55
1,2024-04-07,NYK,2.52,2.50,2.55,MIL,1.56,1.56,1.53
2,2024-04-07,BRK,5.10,4.90,5.00,SAC,1.18,1.19,1.18
3,2024-04-07,SAC,1.18,1.19,1.18,BRK,5.10,4.90,5.00
4,2024-04-07,CHI,6.20,7.50,5.50,ORL,1.11,1.09,1.14
5,2024-04-07,ORL,1.11,1.09,1.14,CHI,6.20,7.50,5.50
6,2024-04-07,CHO,3.60,3.50,3.60,OKC,1.28,1.29,1.29
7,2024-04-07,OKC,1.28,1.29,1.29,CHO,3.60,3.50,3.60
8,2024-04-07,NOP,6.00,5.00,5.00,PHO,1.12,1.16,1.17
9,2024-04-07,PHO,1.12,1.16,1.17,NOP,6.00,5.00,5.00


In [28]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
whole = pd.read_csv(file_path, index_col=0)

In [29]:
complete = pd.concat([whole, final], axis=0)

In [30]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
complete.to_csv(file_path)